In [5]:
# --- CELDA 1: SETUP, INTEGRACIÓN DE DATOS Y GENERACIÓN DE ETIQUETAS ---
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 1. CARGA DE LAS 3 BASES DE DATOS
try:
    df_matricula = pd.read_csv('../data/matricula_senescyt_2015_2023.csv')
    df_ofertas = pd.read_csv('../data/encuentra_empleo_ofertas_2.csv')
    df_inec = pd.read_csv('../data/inec_enemdu_salarios.csv')
    print("✅ Bases de datos cargadas: SENESCYT, Ofertas e INEC.")
except FileNotFoundError:
    print("❌ Error: Faltan archivos CSV. Asegúrate de subirlos.")

# 2. PROCESAMIENTO DE DATOS
# A. Estudiantes: Suma Histórica (Masa total de graduados)
df_estudiantes = df_matricula.groupby('carrera')['num_estudiantes'].sum().reset_index()
df_estudiantes['key'] = df_estudiantes['carrera'].str.lower().str.strip()

# B. Ofertas: Salario promedio y conteo
df_ofertas['salario_oferta'] = (df_ofertas['salario_minimo'] + df_ofertas['salario_maximo']) / 2
df_ofertas_agg = df_ofertas.groupby('carrera_requerida').agg({
    'titulo_puesto': 'count',
    'salario_oferta': 'mean'
}).reset_index()
df_ofertas_agg.rename(columns={'titulo_puesto': 'num_ofertas'}, inplace=True)
df_ofertas_agg['key'] = df_ofertas_agg['carrera_requerida'].str.lower().str.strip()

# C. Función de Mapeo a Sectores INEC (Crucial para relacionar datos)
def mapear_sector(carrera):
    c = carrera.lower()
    if 'sistemas' in c or 'software' in c or 'informática' in c: return 'Información y Comunicación'
    elif 'administración' in c or 'contabilidad' in c or 'financ' in c or 'marketing' in c: return 'Actividades Financieras'
    elif 'medicina' in c or 'enfermería' in c or 'salud' in c or 'odont' in c: return 'Salud Humana'
    elif 'civil' in c or 'arquitectura' in c: return 'Construcción'
    elif 'mecánica' in c or 'industrial' in c or 'eléctrica' in c: return 'Industrias Manufactureras'
    elif 'educación' in c or 'docencia' in c: return 'Educación'
    elif 'agro' in c or 'veterinaria' in c: return 'Agricultura y Ganadería'
    elif 'turismo' in c: return 'Alojamiento y Servicios de Comida'
    else: return 'Actividades Profesionales y Técnicas'

df_estudiantes['sector_economico'] = df_estudiantes['carrera'].apply(mapear_sector)

# D. MERGE TOTAL (Estudiantes + Ofertas + INEC)
df_master = pd.merge(df_estudiantes, df_ofertas_agg, on='key', how='left')

# Preparamos datos INEC (Promedios por sector)
df_inec_clean = df_inec[df_inec['nivel_educacion'] == 'Educación Superior Universitaria']
df_inec_agg = df_inec_clean.groupby('sector_economico').agg({
    'tasa_empleo_formal': 'mean',
    'salario_promedio_mensual': 'mean'
}).reset_index()

df_master = pd.merge(df_master, df_inec_agg, on='sector_economico', how='left')

# Limpieza de Nulos
df_master['num_ofertas'] = df_master['num_ofertas'].fillna(0)
# Si no hay salario de oferta, usamos el referencial del INEC
df_master['salario_oferta'] = df_master['salario_oferta'].fillna(df_master['salario_promedio_mensual'])
df_master['tasa_empleo_formal'] = df_master['tasa_empleo_formal'].fillna(40.0) # Valor por defecto conservador

# 3. GENERACIÓN DE CLUSTERS Y ETIQUETADO (Aquí se crean las etiquetas)
features = ['num_estudiantes', 'num_ofertas', 'salario_oferta', 'tasa_empleo_formal']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_master[features])

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_master['cluster'] = kmeans.fit_predict(X_scaled)

# Lógica de Etiquetado Inteligente
centroides = df_master.groupby('cluster')[features].mean()
medias = df_master[features].mean()

def etiqueta_inteligente(row, m):
    alta_demanda = row['num_ofertas'] > m['num_ofertas']
    buen_sueldo = row['salario_oferta'] > m['salario_oferta']
    empleo_estable = row['tasa_empleo_formal'] > m['tasa_empleo_formal']
    muchos_graduados = row['num_estudiantes'] > m['num_estudiantes']

    if alta_demanda and buen_sueldo: return "En Demanda"
    elif muchos_graduados and (not alta_demanda or not empleo_estable): return "Saturada"
    elif not muchos_graduados and (buen_sueldo or empleo_estable): return "Nicho"
    else: return "Balanceada"

mapa_labels = {i: etiqueta_inteligente(centroides.loc[i], medias) for i in centroides.index}
df_master['categoria'] = df_master['cluster'].map(mapa_labels)

# Seleccionamos columnas finales para el recomendador
df_final = df_master[['carrera', 'sector_economico', 'categoria', 'salario_oferta', 'num_ofertas']].copy()

print(f"📊 Datos Listos. Carreras procesadas: {len(df_final)}")
print("Distribución de etiquetas generadas:\n", df_final['categoria'].value_counts())

✅ Bases de datos cargadas: SENESCYT, Ofertas e INEC.
📊 Datos Listos. Carreras procesadas: 21
Distribución de etiquetas generadas:
 categoria
En Demanda    10
Nicho          8
Balanceada     3
Name: count, dtype: int64


d:\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [7]:
# --- CELDA 2: ENTRENAMIENTO DEL MOTOR NLP ---

# 1. Enriquecimiento del Perfil
# Concatenamos Carrera + Sector INEC para que la IA entienda el contexto
# Ej: "Sistemas" + "Información y Comunicación"
def crear_perfil_busqueda(row):
    texto = f"{row['carrera']} {row['sector_economico']}".lower()
    
    # Añadimos sinónimos "hardcoded" para mejorar la búsqueda humana
    sinonimos = {
        'información': 'tecnología software digital programacion',
        'financieras': 'negocios dinero banca economia gerencia',
        'salud': 'medicina clinica hospital cuidado bienestar',
        'construcción': 'obra infraestructura diseño edificacion',
        'manufactureras': 'produccion fabrica industria procesos'
    }
    
    for key, val in sinonimos.items():
        if key in row['sector_economico'].lower():
            texto += " " + val
            
    return texto

df_final['perfil_nlp'] = df_final.apply(crear_perfil_busqueda, axis=1)

# 2. Vectorización TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_final['perfil_nlp'])

# 3. Matriz de Similitud
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("✅ Motor de Recomendación Semántica entrenado.")

✅ Motor de Recomendación Semántica entrenado.


In [9]:
# --- CELDA 3: FUNCIÓN DE RECOMENDACIÓN FINAL ---

def recomendar_carrera(consulta, filtrar_alta_demanda=False):
    """
    Recomendador semántico basado en Carrera + Sector Económico (INEC).
    """
    print(f"🔍 Analizando perfil para: '{consulta}'...")
    
    # 1. Convertir consulta del usuario a Vector
    try:
        query_vec = tfidf.transform([consulta.lower()])
    except:
        return "Error en el procesamiento de texto."
        
    # 2. Calcular Similitud con toda la base
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
    # 3. Validación
    if sim_scores.max() == 0:
        return pd.DataFrame([["No se encontraron coincidencias. Intenta términos más generales."]], columns=["Mensaje"])

    # 4. Obtener Top Resultados
    top_indices = sim_scores.argsort()[::-1][:10]
    
    # 5. Formatear Salida
    resultados = df_master.iloc[top_indices][['carrera', 'sector_economico', 'categoria', 'salario_oferta']].copy()
    resultados['Similitud'] = (sim_scores[top_indices] * 100).round(1).astype(str) + '%'
    resultados.rename(columns={'sector_economico': 'Sector (INEC)', 'salario_oferta': 'Salario Ref.'}, inplace=True)
    
    # 6. Filtro Opcional (Requisito del Proyecto)
    if filtrar_alta_demanda:
        resultados = resultados[resultados['categoria'] == "En Demanda"]
        if resultados.empty:
            print("⚠️ Se encontraron carreras afines, pero ninguna clasifica como 'Alta Demanda' actualmente.")
            return None

    return resultados.head(5)

# --- PRUEBAS DEL DÍA 6 ---
print("\n--- PRUEBA 1: Búsqueda por Concepto (ej: 'Dinero' o 'Negocios') ---")
display(recomendar_carrera("negocios y finanzas"))

print("\n--- PRUEBA 2: Búsqueda Técnica (ej: 'Construcción') con Filtro de Demanda ---")
display(recomendar_carrera("construcción de obras", filtrar_alta_demanda=False))


--- PRUEBA 1: Búsqueda por Concepto (ej: 'Dinero' o 'Negocios') ---
🔍 Analizando perfil para: 'negocios y finanzas'...


,carrera,Sector (INEC),categoria,Salario Ref.,Similitud
16,Marketing,Actividades Financieras,En Demanda,1128.317121,35.3%
5,Contabilidad y Auditoría,Actividades Financieras,En Demanda,1066.871795,32.3%
0,Administración de Empresas,Actividades Financieras,En Demanda,1199.380597,32.3%
20,Turismo,Alojamiento y Servicios de Comida,Balanceada,632.000000,0.0%
19,Psicología,Actividades Profesionales y Técnicas,Nicho,1216.250000,0.0%



--- PRUEBA 2: Búsqueda Técnica (ej: 'Construcción') con Filtro de Demanda ---
🔍 Analizando perfil para: 'construcción de obras'...


,carrera,Sector (INEC),categoria,Salario Ref.,Similitud
2,Arquitectura,Construcción,Balanceada,876.000000,39.9%
11,Ingeniería Civil,Construcción,En Demanda,1473.290984,38.3%
18,Odontología,Salud Humana,Nicho,1172.750000,0.0%
19,Psicología,Actividades Profesionales y Técnicas,Nicho,1216.250000,0.0%
20,Turismo,Alojamiento y Servicios de Comida,Balanceada,632.000000,0.0%
